In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown,display


In [2]:
OLLAMA_API="http://localhost:11434/api/chat"
MODEL="llama3.2"
HEADERS = {"Content-Type": "application/json"}


In [3]:
class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=HEADERS)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(['script','img','input','style']):
            irrelevant.decompose()
        self.text= soup.body.get_text(separator="\n",strip=True)

In [4]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown and in tamil and english."

In [5]:
def user_prompt_for(website):
    user_prompt=f"You are looking at the website {website.title}"
    user_prompt+="\nThe contents of the website are provided below;Please provide a short summary of the website in markdown in tamil and english.\n\n"
    user_prompt+=website.text
    return user_prompt

In [6]:
def messages_for(website):
      return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
def payload_for(website):
    return {
         "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }


In [12]:
def summarize(url):
    website = Website(url)
    response = requests.post(OLLAMA_API, json=payload_for(website), headers=HEADERS)
    result = response.json()
    return result["message"]["content"]



In [ ]:
summary = summarize("https://edwarddonner.com")
display(Markdown(summary))